In [1]:
from pyvis.network import Network

def visualize_entities(entities, notebook: bool = True):
    """
    Create an interactive graph visualization.
    
    Args:
        entities: Entities object
        notebook: True for Jupyter, False for standalone HTML
    """
    net = Network(
        height="750px", 
        width="100%", 
        bgcolor="#222222", 
        font_color="white",
        notebook=notebook
    )
    
    # Generate colors dynamically for entity types
    entity_types = list(set(e.entity_type for e in entities.entities))
    colors = ['#ff6b6b', '#4ecdc4', '#45b7d1', '#f9ca24', '#6c5ce7', '#95a5a6', '#e17055', '#74b9ff']
    type_colors = dict(zip(entity_types, colors))
    type_colors['unknown'] = '#666666'
    
    # Track which nodes we've added
    added_nodes = set()
    
    # Add nodes for all entities
    for entity in entities.entities:
        
        title_parts = [f"Type: {entity.entity_type}"]
        
        net.add_node(
            entity.name,
            label=entity.name,
            title='\n'.join(title_parts),
            color=type_colors.get(entity.entity_type, type_colors['unknown'])
        )
        added_nodes.add(entity.name)
    
    # Add edges with labels, creating missing nodes as needed
    for entity in entities.entities:
        for rel in entity.relationships:
            # Create target node if it doesn't exist
            if rel.target not in added_nodes:
                net.add_node(
                    rel.target,
                    label=rel.target,
                    title="Type: unknown",
                    color=type_colors['unknown']
                )
                added_nodes.add(rel.target)
            
            
            edge_title = rel.relation

            net.add_edge(
                entity.name,
                rel.target,
                label=rel.relation,
                title=edge_title,
                arrows="to" if rel.directional else None
            )
    
    # Physics settings for better layout
    net.set_options("""
    {
      "physics": {
        "forceAtlas2Based": {
          "gravitationalConstant": -50,
          "springLength": 100,
          "springConstant": 0.08
        },
        "maxVelocity": 50,
        "solver": "forceAtlas2Based"
      }
    }
    """)
    
    return net.show("graph.html")

# 2 - Structured Generation Tutorial

## Introduction
In this tutorial we will build a simple agent that builds knowledge graphs based on some text such as a news article. 

**Knowledge graphs** represent information as entities (nodes) and their relationships (edges). For example, in "Apple acquired Beats in 2014", we'd extract entities like `Apple (Organisation)` and `Beats (Organization)` with a relationship `acquired`. This structured format makes it easy to query, visualize, and reason about complex information.

**Structured generation** ensures LLM outputs match a specific schema. Instead of parsing freeform text (which is error-prone!), we define Python classes and Yera guarantees the LLM returns valid instances. This is perfect for extracting structured data from unstructured text.

We start by importing Yera:

In [2]:
import yera as yr

and then define our system prompt. 

The system prompt instructs the model on what to extract and how to structure it. We provide clear rules about entity types, relationship directions, and extraction guidelines to ensure consistent, high-quality knowledge graphs.

In [3]:
SYS_PROMPT = """You are a knowledge graph extractor.

Extract key entities and their relationships from the text as JSON.

ENTITY TYPES:
- Use single-word types that fit the domain (e.g., Person, Organization, Location, Species, Breed, Product, etc.)
- Common types: Person, Organization, Location, Event, Concept, Law
- Be specific when relevant: "Cat" instead of just "Animal"
- Always use PascalCase or snake_case (no spaces)

RELATIONSHIP RULES:
1. Direction matters - think carefully about who/what is the SUBJECT and OBJECT:
   - CORRECT: John_Smith -> father_of -> Jane_Smith (John IS the father)
   - WRONG: Jane_Smith -> father_of -> John_Smith
   - CORRECT: A -> destroyed -> B (A performed the destruction)
   - WRONG: B -> destroyed -> A (if B destroyed A)

2. Use ACTIVE voice relations (avoid "_of" when ambiguous):
   - GOOD: "worshipped", "employed_by", "located_in", "defeated", "created"
   - AVOID: "worship_of" (ambiguous direction)
   - Use "_by" suffix for passive: "created_by", "defeated_by", "employed_by"
   
3. NO bidirectional relationships unless truly symmetric:
   - "married_to" ✓ (symmetric)
   - "allied_with" ✓ (symmetric)  
   - "destroyed" / "destroyed_by" ✗ (choose ONE direction)

4. Only extract relationships explicitly stated or strongly implied in the text
5. Each entity should have 1-3 key relationships (not exhaustive)
6. Each entity should appear ONCE in the output

EXTRACTION RULES:
- Extract 5-15 most important entities (NO DUPLICATES)
- Focus on entities central to the narrative
- Keep entity names concise (no descriptions in the name field)
- Relationship names should be snake_case verbs or verb phrases

Extract entities and relationships from the following text:"""

## Yera Structs

We define our structures using Yera Struct subclasses. These are based on Pydantic BaseModel with extra features for LLM interaction. The `.fill()` method handles prompt engineering and parsing automatically - you just get a valid Python object back.

Note the validation constraints: `entity_type` must be alphanumeric with underscores (enforced via regex pattern), relationships must have at least one entry (`min_length=1`), and we cap total entities at 50 to respect token limits.

In [4]:

class Relationship(yr.Struct):
    target: str  
    relation: str   
    directional: bool

class Entity(yr.Struct):
    name: str  
    entity_type: str = yr.Field(pattern=r'^[a-zA-Z0-9_]+$')
    relationships: list[Relationship] = yr.Field(min_length=1) 

class KnowledgeGraph(yr.Struct):
    entities: list[Entity] = yr.Field(max_length=50) 

## Building your Agent

The `@yr.agent` decorator (covered in Tutorial 1) wraps your function to handle LLM calls and ensure type-safe returns. You get a validated `KnowledgeGraph` instance back - guaranteed to match your schema.

You can supply your system prompt in the agent decorator, or in the function body with `yr.sys_prompt`. In this case we will use the decorator argument. 

Also, because we are not specifying the LLM to use here Yera will use your configured default. 

In [5]:
@yr.agent(sys_prompt=SYS_PROMPT)
def entity_extractor(text: str) -> KnowledgeGraph:
    return KnowledgeGraph.fill(text)

## Using your Agent

Now let's test our agent. We'll start with a simple sentence to see how it extracts entities and relationships. The agent will parse the text, identify key entities, and structure them according to our schema.

### Simple Example

Let's start with a simple sentence:

In [6]:
simple_text = "John Smith has two cats called Luke and Leia"
simple_result = entity_extractor(simple_text)

╭────────────────────────────────── Startup ───────────────────────────────────╮
│ Started: 2026-02-24 17:52:46                                                 │
│ Top-level Agent: entity_extractor                                            │
╰──────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── entity_extractor ──────────────────────────────╮
│ Identifier:  __main__.entity_extractor                                       │
╰──────────────────────────────────────────────────────────────────────────────╯

You are a knowledge graph extractor.

Extract key entities and their relationships from the text as JSON.

ENTITY TYPES:
- Use single-word types that fit the domain (e.g., Person, Organization, 
Location, Species, Breed, Product, etc.)
- Common types: Person, Organization, Location, Event, Concept, Law
- Be specific when relevant: "Cat" instead of just "Animal"
- Always use PascalCase or snake_case (no spaces)

RELATIONSHIP RUL

and now we visualise the extracted knowledge graph. You should see John Smith (person) connected to Luke and Leia (cats) with appropriate relationships:

In [7]:
visualize_entities(simple_result)

graph.html


### A More Complicated Example

That was straightforward, but knowledge graphs really shine with complex, interconnected information. Let's try a real BBC news article with multiple entities, events, and relationships. The same agent code handles this complexity automatically - it will extract the key players, organizations, locations, and how they all relate to each other.

In [8]:
from pathlib import Path
with open(Path.cwd() / "data" / "articles" / "bbc-news-2.txt", "r") as f:
    res = entity_extractor(f.read())    

╭────────────────────────────────── Startup ───────────────────────────────────╮
│ Started: 2026-02-24 17:54:08                                                 │
│ Top-level Agent: entity_extractor                                            │
╰──────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── entity_extractor ──────────────────────────────╮
│ Identifier:  __main__.entity_extractor                                       │
╰──────────────────────────────────────────────────────────────────────────────╯

You are a knowledge graph extractor.

Extract key entities and their relationships from the text as JSON.

ENTITY TYPES:
- Use single-word types that fit the domain (e.g., Person, Organization, 
Location, Species, Breed, Product, etc.)
- Common types: Person, Organization, Location, Event, Concept, Law
- Be specific when relevant: "Cat" instead of just "Animal"
- Always use PascalCase or snake_case (no spaces)

RELATIONSHIP RUL

Now visualize the full article's knowledge graph. Notice how much richer the connections are - multiple entity types, directional relationships, and a complete picture of the narrative:

In [9]:
visualize_entities(res)

graph.html
